In [4]:
colab = 1
if colab :
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/Github/Opti-Project/src/baselines

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Github/Opti-Project/src/baselines


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
import os
from pathlib import Path
needed_file = Path(os.path.dirname(os.getcwd())) / "vision_experiments"
sys.path.insert(1,str(needed_file))

In [7]:
import random
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
import functools
from train_utils import *
from pathlib import Path
from torch.optim.lr_scheduler import LambdaLR, ReduceLROnPlateau

In [8]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#32 42 73
set_seed(42)

cuda:0


In [9]:
class LogisticRegression(nn.Module):
    """
    Logistic Regression model
    """
    def __init__(self,input_size,num_classes):
        super(LogisticRegression,self).__init__()
        self.linear = nn.Linear(input_size,num_classes)

    def forward(self,feature):
        output = self.linear(feature)
        return output

In [10]:
train_loader, val_loader, test_loader = get_data("MNIST")

Train dataset: 54000 batch size 64 of dim torch.Size([1, 28, 28])
Validation dataset: 6000 batch size 64 of dim torch.Size([1, 28, 28])
Test dataset: 10000 batch size 64 of dim torch.Size([1, 28, 28])


In [11]:
optimizers = {"SGD" : "torch.optim.SGD(model.parameters(), lr=2e-4,nesterov=False,momentum=0.0,weight_decay=0.0)",
     "MomentumSGD": "torch.optim.SGD(model.parameters(), lr=2e-4,nesterov=False,momentum=0.9,weight_decay=0.0)",
     "ClippedSGD" : "torch.optim.SGD(model.parameters(), lr=2e-4,nesterov=False,momentum=0.0,weight_decay=0.0)",
     "Adagrad": "torch.optim.Adagrad(model.parameters(), lr=2e-4, weight_decay=0.0)",
    "Adam" : "torch.optim.Adam(model.parameters(),lr=2e-4, eps= 1e-4, weight_decay= 0.0)"
    }

def try_descent_strategies(optimizers, forward) :
  for opt in optimizers :
    nb_epochs = 30
    input_size = train_loader.dataset[0][0].nelement()
    num_classes = 10
    model = LogisticRegression(input_size,num_classes)
    model.to(device)
    model_name = "logistic_reg" + opt
    print(model_name)
    optimizer = eval(optimizers[opt])
    k_value = 1e-4
    lr_lambda = functools.partial(exponential_lr_decay, k=k_value)
    scheduler = LambdaLR(optimizer, lr_lambda)
    #scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
    output_path = Path("logistic_reg_runs/forward/") if forward else Path("logistic_reg_runs/backward/")
    clip_val = 1 if opt=="ClippedSGD" else 0
    if forward :
      _, _, _, _, run_path = train_validate_forward(
        device,
        nb_epochs,
        model,
        model_name,
        input_size,
        train_loader,
        val_loader,
        optimizer,
        scheduler,
        output_path,
        gradient_clip_val=clip_val)
    else :
        _, _, _, _, run_path = train_validate_backward(
        device,
        nb_epochs,
        model,
        model_name,
        input_size,
        train_loader,
        val_loader,
        optimizer,
        scheduler,
        output_path,
        gradient_clip_val=clip_val)
    test_results = evaluate(device, model, input_size, test_loader, run_path=run_path)
    print("test results")
    print(test_results)


# Backward

In [ ]:
try_descent_strategies(optimizers, forward=False)

logistic_regSGD


	Validating: 100%|██████████| 94/94 [00:01<00:00, 59.23it/s, iters_per_sec=59.2, vacc=0.812, vf1=0.791, vloss=1.16]


Epoch 1 lr: 0.0002 Averages : Train Loss: 1.7955 Val Loss: 1.3245 Train Acc: 0.4577 Val Acc: 0.6966 Train F1: 0.4194 Val F1: 0.6580
Best model saved


	Validating: 100%|██████████| 94/94 [00:01<00:00, 58.70it/s, iters_per_sec=58.7, vacc=0.729, vf1=0.665, vloss=1.06]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 1.1178 Val Loss: 0.9633 Train Acc: 0.7468 Val Acc: 0.7854 Train F1: 0.7149 Val F1: 0.7615
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 0.2444 Val Loss: 0.2920 Train Acc: 0.9324 Val Acc: 0.9199 Train F1: 0.9247 Val F1: 0.9112


	Validating: 100%|██████████| 94/94 [00:01<00:00, 55.95it/s, iters_per_sec=55.9, vacc=0.938, vf1=0.94, vloss=0.324]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 0.2436 Val Loss: 0.2911 Train Acc: 0.9328 Val Acc: 0.9177 Train F1: 0.9252 Val F1: 0.9071


	Evaluating: 100%|██████████| 157/157 [00:02<00:00, 57.60it/s, eval_acc=0.938, eval_f1=0.933, iters_per_sec=57.6]

test results
{'Acc': 0.9273487261146497, 'F1': 0.9183748558727158}


# Forward

In [22]:
try_descent_strategies(optimizers, forward=True)

logistic_regSGD


	Validating: 100%|██████████| 94/94 [00:01<00:00, 54.79it/s, iters_per_sec=54.8, vacc=0.625, vf1=0.517, vloss=1.39]


Epoch 1 lr: 0.0002 Averages : Train Loss: 1.8978 Val Loss: 1.4612 Train Acc: 0.3731 Val Acc: 0.5716 Train F1: 0.3327 Val F1: 0.5387
Best model saved


	Validating: 100%|██████████| 94/94 [00:01<00:00, 53.77it/s, iters_per_sec=53.7, vacc=0.708, vf1=0.634, vloss=1.14]


Epoch 2 lr: 0.00019998000099996668 Averages : Train Loss: 1.2292 Val Loss: 1.0875 Train Acc: 0.6471 Val Acc: 0.6907 Train F1: 0.6126 Val F1: 0.6610
Best model saved


...

Epoch 29 lr: 0.0001994407832687786 Averages : Train Loss: 0.4988 Val Loss: 0.5088 Train Acc: 0.8610 Val Acc: 0.8512 Train F1: 0.8467 Val F1: 0.8349
Best model saved


	Validating: 100%|██████████| 94/94 [00:01<00:00, 55.03it/s, iters_per_sec=55, vacc=0.938, vf1=0.94, vloss=0.368]


Epoch 30 lr: 0.0001994208401876224 Averages : Train Loss: 0.4926 Val Loss: 0.5015 Train Acc: 0.8623 Val Acc: 0.8539 Train F1: 0.8486 Val F1: 0.8369
Best model saved


	Evaluating: 100%|██████████| 157/157 [00:02<00:00, 57.29it/s, eval_acc=0.812, eval_f1=0.747, iters_per_sec=57.3]


test results
{'Acc': 0.8706210191082803, 'F1': 0.8571359671919306}


# Plots

In [12]:
import pickle

run_path_backward = {"SGD":"logistic_regSGDModel_start_13_06_2024_start_16h40m/",
"MomentumSGD":"logistic_regMomentumSGDModel_start_13_06_2024_start_16h48m/",
"ClippedSGD":"logistic_regClippedSGDModel_start_13_06_2024_start_16h57m/",
"Adagrad":"logistic_regAdagradModel_start_13_06_2024_start_17h05m/",
"Adam":"logistic_regAdamModel_start_13_06_2024_start_17h14m/"}

run_path_forward = {"SGD":"logistic_regSGDModel_start_13_06_2024_start_18h16m/",
"MomentumSGD":"logistic_regMomentumSGDModel_start_13_06_2024_start_18h25m/",
"ClippedSGD":"logistic_regClippedSGDModel_start_13_06_2024_start_18h35m/",
"Adagrad":"logistic_regAdagradModel_start_13_06_2024_start_18h44m/",
"Adam":"logistic_regAdamModel_start_13_06_2024_start_18h53m/"}

metrics = ["Loss","Acc","F1","Time"]
train_run_epochs = {"forward" : {opt : {k:[] for k in metrics}for opt in optimizers.keys()} , "backward"  : {opt : {k:[] for k in metrics}for opt in optimizers.keys()}}
train_run_its = {"forward" : {opt : {k:[] for k in metrics} for opt in optimizers.keys()}, "backward" : {opt : {k:[] for k in metrics}for opt in optimizers.keys()}}
val_run_epochs = {"forward" : {opt : {k:[] for k in metrics} for opt in optimizers.keys()}, "backward" : { opt : {k:[] for k in metrics}for opt in optimizers.keys()}}
val_run_its = {"forward" : {opt : {k:[] for k in metrics} for opt in optimizers.keys()}, "backward" : {opt : {k:[] for k in metrics}for opt in optimizers.keys()}}

In [13]:
for opt in run_path_backward :
    for strat in ["forward", "backward"] :
        run_path = "logistic_reg_runs/forward/"+run_path_forward[opt] if strat=="forward" else "logistic_reg_runs/backward/"+run_path_backward[opt]
        for k in metrics:
          train_run_epochs[strat][opt][k] = pickle.load(open(run_path + "{}_train_run.pkl".format(k), "rb"))
          train_run_its[strat][opt][k] = pickle.load(open(run_path + "{}_train_run_its.pkl".format(k), "rb"))
          val_run_epochs[strat][opt][k] = pickle.load(open(run_path + "{}_val_run.pkl".format(k), "rb"))
          val_run_its[strat][opt][k] = pickle.load(open(run_path + "{}_val_run_its.pkl".format(k), "rb"))